In [ ]:
import unittest
import time
import sys
import os
import traceback
import requests

from bs4 import BeautifulSoup, Comment
from lxml import etree

#test

def load_mlb_schedule(year):
    
    BASE_URL = "https://www.baseball-reference.com" 
    link = f"{BASE_URL}/leagues/MLB/{year}-schedule.shtml"
    schedule = []
    try:
        #crear el repots
        os.mkdir(str(year))

        #carga el archivo de los datos
        #file = open("mlb_game.txt","a")

        # cargar la pagina
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
        
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            print(f"Schedule {toc - tic+1:0.4} seconds url: {link}")
            soup = BeautifulSoup(html,'html.parser')
            div_section_wrapper = soup.find_all('div',class_= 'section_wrapper')
            div_section_content = div_section_wrapper[0].find_all('div',class_='section_content')
            divs = div_section_content[0].find_all('div')
            print(len(divs))
            for div in divs:
                ps = div.find_all('p', class_="game")
                for p in ps:
                    ems = p.find_all('em')
                    for em in ems:
                        schedule.append(f"{BASE_URL}{em.a.get('href')}")
        return schedule
    except ValueError:
            print("Error de Valor")
            traceback.print_exception(*sys.exc_info())
            return schedule
    except:    
            print("Error de general") 
            traceback.print_exception(*sys.exc_info())   
            return schedule                     

def load_mlb_games(link,folder):
    

    try:

        #carga el archivo de los datos
        file = open(str(folder)+'//'+link.split('/')[5],'a')

        # cargar la pagina
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
        
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            print(f"Process {toc - tic+1:0.4} seconds url: {link}")
            soup = BeautifulSoup(html,'html.parser')
            comments = soup.find_all(text=lambda text:isinstance(text, Comment))
            htm = str([htm for htm in comments if "div_play_by_play" in htm]);
            if(htm):                          
                div = BeautifulSoup(htm,'html.parser')
                tbody  = div.find_all("tbody")
                rows = tbody[0].find_all('tr', class_ =['top_inning','bottom_inning'])
                #print(rows)
                for row in rows:
                    if row != [''] :
                       cols=row.find_all('td')
                       cols=[" ".join(x.text.replace(',','|').upper().split()) for x in cols]
                       file.write(" , ".join(cols)+'\n')
            file.close  
    except ValueError:
            file.close 
            print("Error de Valor")
            traceback.print_exception(*sys.exc_info())
    except:    
            file.close                     
            print("Error de general") 
            traceback.print_exception(*sys.exc_info())   

#load_mlb_schedule(2018) 
#load_mlb_games('https://www.baseball-reference.com/boxes/CHN/CHN201810020.shtml')
                                
for score in load_mlb_schedule(2018):
    load_mlb_games(score,2018)
                                
                               
                                
                            

Schedule 1.37 seconds url: https://www.baseball-reference.com/leagues/MLB/2018-schedule.shtml
184
Process 1.326 seconds url: https://www.baseball-reference.com/boxes/ARI/ARI201803290.shtml
Process 1.526 seconds url: https://www.baseball-reference.com/boxes/ATL/ATL201803290.shtml
Process 1.893 seconds url: https://www.baseball-reference.com/boxes/BAL/BAL201803290.shtml
Process 1.489 seconds url: https://www.baseball-reference.com/boxes/KCA/KCA201803290.shtml
Process 1.409 seconds url: https://www.baseball-reference.com/boxes/LAN/LAN201803290.shtml
Process 1.402 seconds url: https://www.baseball-reference.com/boxes/MIA/MIA201803290.shtml
Process 1.357 seconds url: https://www.baseball-reference.com/boxes/NYN/NYN201803290.shtml
Process 1.414 seconds url: https://www.baseball-reference.com/boxes/OAK/OAK201803290.shtml
Process 1.316 seconds url: https://www.baseball-reference.com/boxes/SDN/SDN201803290.shtml
Process 1.348 seconds url: https://www.baseball-reference.com/boxes/SEA/SEA20180329

Process 1.33 seconds url: https://www.baseball-reference.com/boxes/MIN/MIN201804050.shtml
Process 1.318 seconds url: https://www.baseball-reference.com/boxes/NYA/NYA201804050.shtml
Process 1.366 seconds url: https://www.baseball-reference.com/boxes/OAK/OAK201804050.shtml
Process 1.37 seconds url: https://www.baseball-reference.com/boxes/PHI/PHI201804050.shtml
Process 1.326 seconds url: https://www.baseball-reference.com/boxes/PIT/PIT201804050.shtml
Process 1.37 seconds url: https://www.baseball-reference.com/boxes/SDN/SDN201804050.shtml
Process 1.381 seconds url: https://www.baseball-reference.com/boxes/SLN/SLN201804050.shtml
Process 1.327 seconds url: https://www.baseball-reference.com/boxes/WAS/WAS201804050.shtml
Process 1.399 seconds url: https://www.baseball-reference.com/boxes/ANA/ANA201804060.shtml
Process 1.284 seconds url: https://www.baseball-reference.com/boxes/CLE/CLE201804060.shtml
Process 1.309 seconds url: https://www.baseball-reference.com/boxes/COL/COL201804060.shtml
Pr

Process 1.382 seconds url: https://www.baseball-reference.com/boxes/KCA/KCA201804120.shtml
Process 1.313 seconds url: https://www.baseball-reference.com/boxes/MIN/MIN201804120.shtml
Process 1.325 seconds url: https://www.baseball-reference.com/boxes/SDN/SDN201804120.shtml
Process 1.502 seconds url: https://www.baseball-reference.com/boxes/WAS/WAS201804120.shtml
Process 1.28 seconds url: https://www.baseball-reference.com/boxes/BOS/BOS201804130.shtml
Process 1.375 seconds url: https://www.baseball-reference.com/boxes/CHN/CHN201804130.shtml
Process 1.335 seconds url: https://www.baseball-reference.com/boxes/CIN/CIN201804130.shtml
Process 1.334 seconds url: https://www.baseball-reference.com/boxes/CLE/CLE201804130.shtml
Process 1.358 seconds url: https://www.baseball-reference.com/boxes/DET/DET201804130.shtml
Process 1.376 seconds url: https://www.baseball-reference.com/boxes/HOU/HOU201804130.shtml
Process 1.307 seconds url: https://www.baseball-reference.com/boxes/KCA/KCA201804130.shtml


Process 1.423 seconds url: https://www.baseball-reference.com/boxes/CHA/CHA201804200.shtml
Process 1.311 seconds url: https://www.baseball-reference.com/boxes/COL/COL201804200.shtml
Process 1.304 seconds url: https://www.baseball-reference.com/boxes/DET/DET201804201.shtml
Process 1.35 seconds url: https://www.baseball-reference.com/boxes/DET/DET201804202.shtml
Process 1.313 seconds url: https://www.baseball-reference.com/boxes/LAN/LAN201804200.shtml
Process 1.326 seconds url: https://www.baseball-reference.com/boxes/MIL/MIL201804200.shtml
Process 1.391 seconds url: https://www.baseball-reference.com/boxes/NYA/NYA201804200.shtml
Process 1.319 seconds url: https://www.baseball-reference.com/boxes/OAK/OAK201804200.shtml
Process 1.319 seconds url: https://www.baseball-reference.com/boxes/PHI/PHI201804200.shtml
Process 1.326 seconds url: https://www.baseball-reference.com/boxes/SLN/SLN201804200.shtml
Process 1.337 seconds url: https://www.baseball-reference.com/boxes/TBA/TBA201804200.shtml


Process 1.403 seconds url: https://www.baseball-reference.com/boxes/BOS/BOS201804270.shtml
Process 1.309 seconds url: https://www.baseball-reference.com/boxes/CHN/CHN201804270.shtml
Process 1.337 seconds url: https://www.baseball-reference.com/boxes/CLE/CLE201804270.shtml
Process 1.375 seconds url: https://www.baseball-reference.com/boxes/HOU/HOU201804270.shtml
Process 1.354 seconds url: https://www.baseball-reference.com/boxes/KCA/KCA201804270.shtml
Process 1.311 seconds url: https://www.baseball-reference.com/boxes/MIA/MIA201804270.shtml
Process 1.332 seconds url: https://www.baseball-reference.com/boxes/MIN/MIN201804270.shtml
Process 1.311 seconds url: https://www.baseball-reference.com/boxes/PHI/PHI201804270.shtml
Process 1.335 seconds url: https://www.baseball-reference.com/boxes/PIT/PIT201804270.shtml
Process 1.303 seconds url: https://www.baseball-reference.com/boxes/SDN/SDN201804270.shtml
Process 1.359 seconds url: https://www.baseball-reference.com/boxes/SFN/SFN201804270.shtml

...

Traceback (most recent call last):
  File "<ipython-input-12-eb6973d8a3aa>", line 65, in load_mlb_games
    time.sleep(3)
KeyboardInterrupt


Process 1.345 seconds url: https://www.baseball-reference.com/boxes/CLE/CLE201806060.shtml
Process 1.354 seconds url: https://www.baseball-reference.com/boxes/HOU/HOU201806060.shtml
Error de general


Traceback (most recent call last):
  File "<ipython-input-12-eb6973d8a3aa>", line 78, in load_mlb_games
    div = BeautifulSoup(htm,'html.parser')
  File "C:\Anaconda3\lib\site-packages\bs4\__init__.py", line 281, in __init__
    self._feed()
  File "C:\Anaconda3\lib\site-packages\bs4\__init__.py", line 342, in _feed
    self.builder.feed(self.markup)
  File "C:\Anaconda3\lib\site-packages\bs4\builder\_htmlparser.py", line 247, in feed
    parser.feed(markup)
  File "C:\Anaconda3\lib\html\parser.py", line 111, in feed
    self.goahead(0)
  File "C:\Anaconda3\lib\html\parser.py", line 171, in goahead
    k = self.parse_starttag(i)
  File "C:\Anaconda3\lib\html\parser.py", line 303, in parse_starttag
    endpos = self.check_for_whole_start_tag(i)
  File "C:\Anaconda3\lib\html\parser.py", line 355, in check_for_whole_start_tag
    if m:
KeyboardInterrupt


Error de general


Traceback (most recent call last):
  File "<ipython-input-12-eb6973d8a3aa>", line 65, in load_mlb_games
    time.sleep(3)
KeyboardInterrupt


'CHN201810020.shtml'